In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('./titanic/data/train.csv')

In [6]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [11]:
age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

In [17]:
from scipy.stats import mode

df_nonull = df.dropna()

mode_embarked = mode(df_nonull['Embarked'])[0][0]
df['Embarked'] = df['Embarked'].fillna(mode_embarked)

/home/yli/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [18]:
df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)
df['Port'] = df['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived       891 non-null int64
PassengerId    891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Gender         891 non-null int64
Port           891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB


In [20]:
train_data = df.values

In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100)
model = model.fit(train_data[0:,2:],train_data[0:,0])

In [22]:
df_test = pd.read_csv('./titanic/data/test.csv')

In [23]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [24]:
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [25]:
df_test['Age'] = df_test['Age'].fillna(age_mean)

In [80]:
fare_means = df.pivot_table(index='Pclass', aggfunc='mean')

In [81]:
fare_means['Fare'][3]

13.675550101832997

In [48]:
fare_means.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 1 to 3
Data columns (total 1 columns):
Fare    3 non-null float64
dtypes: float64(1)
memory usage: 48.0 bytes


In [59]:
fare_means.shape

(3, 1)

In [82]:
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x: fare_means['Fare'][x['Pclass']] if pd.isnull(x['Fare']) else x['Fare'], axis = 1)

In [83]:
df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test['Port'] = df_test['Embarked'].map({'C':1, 'S':2, 'Q':3})

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

In [85]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]
df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])

df_result.to_csv('./titanic/data/titanic_1-1.csv', index=False)

In [87]:
df_result.shape

(418, 2)